In [1]:
EXP_NAME = 'pretraining_REMBERT_79'
FOLDER_NAME = EXP_NAME
SEED = 79

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import os

import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler

from datasets import Dataset
from sklearn import model_selection 
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

from tqdm.autonotebook import tqdm

import gc

import re
import random
random.seed(SEED)

import warnings
warnings.filterwarnings("ignore")

In [4]:
model_checkpoint = 'google/rembert'
batch_size = 8
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
max_length = 384
doc_stride = 192
pad_on_right = tokenizer.padding_side == "right"
n_folds = 3

In [5]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [6]:
def create_folds(data, num_splits):
    data["kfold"] = -1
    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=2021)
    for fold_num, (t_, v_) in enumerate(kf.split(X=data, y=data.language.values)):
        data.loc[v_, "kfold"] = fold_num
    return data

def convert_answers(row):
    return {"answer_start": [row[0]], "text": [row[1]]}

In [7]:
def negative_sampling(examples, ratio=0.1):
    
    def _sample(pos):
        if pos != 0: return True
        else: return random.random() < ratio
                
    indices = [i for i,x in enumerate(examples['start_positions']) if _sample(x)]

    for key in examples.keys():
        examples[key] = [x for i,x in enumerate(examples[key]) if i in indices]
        
    return examples

In [8]:
fold = 2

In [9]:
from datasets import load_dataset
squad = load_dataset("squad")

Reusing dataset squad (/home/darek/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
tydi = pd.read_csv('tydiqa_train.csv.zip')
tydi_bete_all = tydi[(tydi.language == 'bengali') | (tydi.language == 'telugu') | (tydi.language == 'english')].reset_index(drop=True)
print(len(tydi_bete_all))
tydi_bete_all.head(1)

11650


,id,context,question,answer_text,answer_start,language
0,6553242570085056655,"\nవేపచెట్టు, వేపాకు, వేపపూత ఇలా వేపచెట్టునుండి...",వేప చెట్టు యొక్క శాస్త్రీయ నామం ఏమిటి?,Azadirachta indica,702,telugu


In [11]:
nq = pd.read_csv('nq_small.csv.zip')
nq = nq.rename(columns={'answer':'answer_text'})
nq["answers"] = nq[["answer_start", "answer_text"]].apply(convert_answers, axis=1)
print(len(nq))
nq.head(1)

10658


,answer_text,context,id,question,answer_start,p0,p1,alen,answers
0,103 Hz,\n\nThe hertz (symbol: Hz) is the derived unit...,532428134682627320,1 kilohertz is equal to how many hertz?,324,-0.469617,0.347219,6,"{'answer_start': [324], 'text': ['103 Hz']}"


In [12]:
train = pd.read_csv("train.csv")
threeds = pd.read_csv('chaii-mlqa-xquad-5folds.csv')
hindi = threeds[threeds.src != 'chaii'].reset_index(drop=True)
chaii = threeds[threeds.src == 'chaii'].reset_index(drop=True)
chaii = pd.merge(chaii, train[['id', 'context', 'question']], how='left', on=['context', 'question'])
del chaii['fold']
chaii = create_folds(chaii, 3)

chaii_train = chaii[chaii.kfold != fold]
chaii_valid = chaii[chaii.kfold == fold]

chaii_train["answers"] = chaii_train[["answer_start", "answer_text"]].apply(convert_answers, axis=1)
chaii_valid["answers"] = chaii_valid[["answer_start", "answer_text"]].apply(convert_answers, axis=1)
hindi["answers"] = hindi[["answer_start", "answer_text"]].apply(convert_answers, axis=1)
tydi_bete_all["answers"] = tydi_bete_all[["answer_start", "answer_text"]].apply(convert_answers, axis=1)

chaii_train_ds = Dataset.from_pandas(chaii_train)
chaii_valid_ds = Dataset.from_pandas(chaii_valid)
hindi_ds = Dataset.from_pandas(hindi)
tydi_bete_all_ds = Dataset.from_pandas(tydi_bete_all)
nq_ds = Dataset.from_pandas(nq)

tokenized_tydi_bete_all_strid = tydi_bete_all_ds.map(prepare_train_features, batched=True, remove_columns=tydi_bete_all_ds.column_names)
tokenized_nq_strid = nq_ds.map(prepare_train_features, batched=True, remove_columns=nq_ds.column_names)
tokenized_chaii_train_strid = chaii_train_ds.map(prepare_train_features, batched=True, remove_columns=chaii_train_ds.column_names)
tokenized_chaii_valid_strid = chaii_valid_ds.map(prepare_train_features, batched=True, remove_columns=chaii_valid_ds.column_names)
tokenized_hindi_strid = hindi_ds.map(prepare_train_features, batched=True, remove_columns=hindi_ds.column_names)
tokenized_squad = squad['train'].map(prepare_train_features, batched=True, remove_columns=squad['train'].column_names)

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/darek/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-9cea1f41c13bebac.arrow


In [13]:
tokenized_tydi_bete_all_strid1 = tokenized_tydi_bete_all_strid.map(negative_sampling, batched=True, batch_size=8)
tokenized_tydi_bete_all_strid2 = tokenized_tydi_bete_all_strid.map(negative_sampling, batched=True, batch_size=8)
tokenized_nq_strid = tokenized_nq_strid.map(negative_sampling, fn_kwargs={'ratio':0.06}, batched=True, batch_size=8)
tokenized_chaii_train_strid1 = tokenized_chaii_train_strid.map(negative_sampling, fn_kwargs={'ratio':0.1}, batched=True, batch_size=8)
tokenized_chaii_train_strid2 = tokenized_chaii_train_strid.map(negative_sampling, fn_kwargs={'ratio':0.2}, batched=True, batch_size=8)

  0%|          | 0/39484 [00:00<?, ?ba/s]

  0%|          | 0/39484 [00:00<?, ?ba/s]

  0%|          | 0/34916 [00:00<?, ?ba/s]

  0%|          | 0/1695 [00:00<?, ?ba/s]

  0%|          | 0/1695 [00:00<?, ?ba/s]

In [14]:
from datasets import concatenate_datasets

ep1 = concatenate_datasets([tokenized_tydi_bete_all_strid1, tokenized_squad, tokenized_hindi_strid, tokenized_chaii_train_strid1]).shuffle(seed=SEED)
ep2 = concatenate_datasets([tokenized_tydi_bete_all_strid2, tokenized_nq_strid, tokenized_hindi_strid, tokenized_chaii_train_strid2]).shuffle(seed=SEED)
tokenized_train_all = concatenate_datasets([ep1, ep2, tokenized_chaii_train_strid.shuffle(seed=SEED)])


In [15]:
len(tokenized_train_all)

249543

In [16]:
from torch.utils.data import DataLoader, Dataset, IterableDataset, RandomSampler, SequentialSampler

from transformers.file_utils import is_datasets_available

class MyTrainer(Trainer): 

    def get_train_dataloader(self) -> DataLoader:
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")

        train_dataset = self.train_dataset
        if is_datasets_available() and isinstance(train_dataset, Dataset):
            train_dataset = self._remove_unused_columns(train_dataset, description="training")

        train_sampler = SequentialSampler(self.train_dataset)

        return DataLoader(
            train_dataset,
            batch_size=self.args.train_batch_size,
            sampler=train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
        )

In [17]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at google/rembert were not used when initializing RemBertForQuestionAnswering: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RemBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RemBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RemBertForQuestionAnswering were not initialized from the model checkpoint at google/rembert and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
args = TrainingArguments(
    f"chaii-qa-{EXP_NAME}",
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    logging_steps = 200,
    save_steps = 200,
    save_strategy = "steps",
    learning_rate=9e-6,
    gradient_accumulation_steps=16,
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to='none',
    save_total_limit=6
)

data_collator = default_data_collator

trainer = MyTrainer(
    model,
    args,
    train_dataset=tokenized_train_all,
    eval_dataset=tokenized_chaii_valid_strid,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [19]:
trainer.train()
trainer.save_model(f"{FOLDER_NAME}/final")

***** Running training *****
  Num examples = 249543
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 16
  Total optimization steps = 1949


Step,Training Loss,Validation Loss
200,2.562000,0.451003
400,0.774600,0.265564
600,0.715000,0.306325
800,0.682300,0.354410
1000,0.649300,0.231298
1200,0.638400,0.216148
1400,0.581400,0.277678
1600,0.548300,0.223556
1800,0.537800,0.244234


***** Running Evaluation *****
  Num examples = 6456
  Batch size = 8
Saving model checkpoint to chaii-qa-pretraining_REMBERT_79/checkpoint-200
Configuration saved in chaii-qa-pretraining_REMBERT_79/checkpoint-200/config.json
Model weights saved in chaii-qa-pretraining_REMBERT_79/checkpoint-200/pytorch_model.bin
tokenizer config file saved in chaii-qa-pretraining_REMBERT_79/checkpoint-200/tokenizer_config.json
Special tokens file saved in chaii-qa-pretraining_REMBERT_79/checkpoint-200/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 6456
  Batch size = 8
Saving model checkpoint to chaii-qa-pretraining_REMBERT_79/checkpoint-400
Configuration saved in chaii-qa-pretraining_REMBERT_79/checkpoint-400/config.json
Model weights saved in chaii-qa-pretraining_REMBERT_79/checkpoint-400/pytorch_model.bin
tokenizer config file saved in chaii-qa-pretraining_REMBERT_79/checkpoint-400/tokenizer_config.json
Special tokens file saved in chaii-qa-pretraining_REMBERT_79/checkpoint-

In [20]:
import collections

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

from string import punctuation

def postuning(s):
    s = " ".join(s.split())
    s = s.strip(punctuation)
    return s

In [21]:
validation_features = chaii_valid_ds.map(
    prepare_validation_features,
    batched=True,
    remove_columns=chaii_valid_ds.column_names
)

valid_feats_small = validation_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

max_answer_length = 30

examples = chaii_valid_ds
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

references = [{"id": ex["id"], "context": ex["context"], "question": ex["question"], "answer": ex["answer_text"]} for ex in chaii_valid_ds]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/6456 [00:00<?, ?ex/s]

In [22]:
# final
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(chaii_valid_ds, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res.jaccard.mean(), res.pjaccard.mean()

***** Running Prediction *****
  Num examples = 6456
  Batch size = 8


Post-processing 370 example predictions split into 6456 features.


  0%|          | 0/370 [00:00<?, ?it/s]

(0.7492985842985843, 0.7686679536679536)

In [23]:
model_checkpoint = 'chaii-qa-pretraining_REMBERT_79/checkpoint-800'
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(chaii_valid_ds, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res.jaccard.mean(), res.pjaccard.mean()

loading configuration file chaii-qa-pretraining_REMBERT_79/checkpoint-800/config.json
Model config RemBertConfig {
  "_name_or_path": "google/rembert",
  "architectures": [
    "RemBertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 312,
  "classifier_dropout_prob": 0.1,
  "embedding_dropout_prob": 0,
  "embedding_size": 256,
  "eos_token_id": 313,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 1152,
  "initializer_range": 0.02,
  "input_embedding_size": 256,
  "intermediate_size": 4608,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "rembert",
  "num_attention_heads": 18,
  "num_hidden_layers": 32,
  "output_embedding_size": 1664,
  "pad_token_id": 0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250300
}

loading weights file chaii-qa-pretraining_REMBERT_79/checkpoint-800/pytorch_model.bin


Post-processing 370 example predictions split into 6456 features.


  0%|          | 0/370 [00:00<?, ?it/s]

(0.7471374946374947, 0.7656059631059631)

In [24]:
model_checkpoint = 'chaii-qa-pretraining_REMBERT_79/checkpoint-1000'
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(chaii_valid_ds, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res.jaccard.mean(), res.pjaccard.mean()

loading configuration file chaii-qa-pretraining_REMBERT_79/checkpoint-1000/config.json
Model config RemBertConfig {
  "_name_or_path": "google/rembert",
  "architectures": [
    "RemBertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 312,
  "classifier_dropout_prob": 0.1,
  "embedding_dropout_prob": 0,
  "embedding_size": 256,
  "eos_token_id": 313,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 1152,
  "initializer_range": 0.02,
  "input_embedding_size": 256,
  "intermediate_size": 4608,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "rembert",
  "num_attention_heads": 18,
  "num_hidden_layers": 32,
  "output_embedding_size": 1664,
  "pad_token_id": 0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250300
}

loading weights file chaii-qa-pretraining_REMBERT_79/checkpoint-1000/pytorch_model.bi

Post-processing 370 example predictions split into 6456 features.


  0%|          | 0/370 [00:00<?, ?it/s]

(0.7537945087945087, 0.7691098241098241)

In [25]:
model_checkpoint = 'chaii-qa-pretraining_REMBERT_79/checkpoint-1200'
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(chaii_valid_ds, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res.jaccard.mean(), res.pjaccard.mean()

loading configuration file chaii-qa-pretraining_REMBERT_79/checkpoint-1200/config.json
Model config RemBertConfig {
  "_name_or_path": "google/rembert",
  "architectures": [
    "RemBertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 312,
  "classifier_dropout_prob": 0.1,
  "embedding_dropout_prob": 0,
  "embedding_size": 256,
  "eos_token_id": 313,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 1152,
  "initializer_range": 0.02,
  "input_embedding_size": 256,
  "intermediate_size": 4608,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "rembert",
  "num_attention_heads": 18,
  "num_hidden_layers": 32,
  "output_embedding_size": 1664,
  "pad_token_id": 0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250300
}

loading weights file chaii-qa-pretraining_REMBERT_79/checkpoint-1200/pytorch_model.bi

Post-processing 370 example predictions split into 6456 features.


  0%|          | 0/370 [00:00<?, ?it/s]

(0.7344251394251393, 0.7488395538395538)

In [26]:
model_checkpoint = 'chaii-qa-pretraining_REMBERT_79/checkpoint-1400'
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(chaii_valid_ds, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res.jaccard.mean(), res.pjaccard.mean()

loading configuration file chaii-qa-pretraining_REMBERT_79/checkpoint-1400/config.json
Model config RemBertConfig {
  "_name_or_path": "google/rembert",
  "architectures": [
    "RemBertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 312,
  "classifier_dropout_prob": 0.1,
  "embedding_dropout_prob": 0,
  "embedding_size": 256,
  "eos_token_id": 313,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 1152,
  "initializer_range": 0.02,
  "input_embedding_size": 256,
  "intermediate_size": 4608,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "rembert",
  "num_attention_heads": 18,
  "num_hidden_layers": 32,
  "output_embedding_size": 1664,
  "pad_token_id": 0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250300
}

loading weights file chaii-qa-pretraining_REMBERT_79/checkpoint-1400/pytorch_model.bi

Post-processing 370 example predictions split into 6456 features.


  0%|          | 0/370 [00:00<?, ?it/s]

(0.7289639639639639, 0.7474324324324323)

In [27]:
model_checkpoint = 'chaii-qa-pretraining_REMBERT_79/checkpoint-1600'
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(chaii_valid_ds, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res.jaccard.mean(), res.pjaccard.mean()

loading configuration file chaii-qa-pretraining_REMBERT_79/checkpoint-1600/config.json
Model config RemBertConfig {
  "_name_or_path": "google/rembert",
  "architectures": [
    "RemBertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 312,
  "classifier_dropout_prob": 0.1,
  "embedding_dropout_prob": 0,
  "embedding_size": 256,
  "eos_token_id": 313,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 1152,
  "initializer_range": 0.02,
  "input_embedding_size": 256,
  "intermediate_size": 4608,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "rembert",
  "num_attention_heads": 18,
  "num_hidden_layers": 32,
  "output_embedding_size": 1664,
  "pad_token_id": 0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250300
}

loading weights file chaii-qa-pretraining_REMBERT_79/checkpoint-1600/pytorch_model.bi

Post-processing 370 example predictions split into 6456 features.


  0%|          | 0/370 [00:00<?, ?it/s]

(0.7447039897039897, 0.7654247104247103)

In [28]:
model_checkpoint = 'chaii-qa-pretraining_REMBERT_79/checkpoint-1800'
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(chaii_valid_ds, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res.jaccard.mean(), res.pjaccard.mean()

loading configuration file chaii-qa-pretraining_REMBERT_79/checkpoint-1800/config.json
Model config RemBertConfig {
  "_name_or_path": "google/rembert",
  "architectures": [
    "RemBertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 312,
  "classifier_dropout_prob": 0.1,
  "embedding_dropout_prob": 0,
  "embedding_size": 256,
  "eos_token_id": 313,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 1152,
  "initializer_range": 0.02,
  "input_embedding_size": 256,
  "intermediate_size": 4608,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "rembert",
  "num_attention_heads": 18,
  "num_hidden_layers": 32,
  "output_embedding_size": 1664,
  "pad_token_id": 0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250300
}

loading weights file chaii-qa-pretraining_REMBERT_79/checkpoint-1800/pytorch_model.bi

Post-processing 370 example predictions split into 6456 features.


  0%|          | 0/370 [00:00<?, ?it/s]

(0.748082368082368, 0.7688030888030888)

In [ ]:
###

In [31]:
!gdrive upload chaii-qa-pretraining_REMBERT_79/checkpoint-1000/pytorch_model.bin

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Uploading chaii-qa-pretraining_REMBERT_79/checkpoint-1000/pytorch_model.bin
Uploaded 1MBqJVu0DNmplM1v-Qlr1aTnU122v1D4F at 3.0 MB/s, total 2.3 GB


In [32]:
!gdrive upload chaii-qa-pretraining_REMBERT_79/checkpoint-1000/config.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Uploading chaii-qa-pretraining_REMBERT_79/checkpoint-1000/config.json
Uploaded 1-zdPMb-8eSaxe4UMiDSp0Zpj-IcMihck at 655.0 B/s, total 800.0 B


In [29]:
!gdrive upload pretraining_REMBERT_79/final/pytorch_model.bin

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Uploading pretraining_REMBERT_79/final/pytorch_model.bin
Uploaded 1LcQNU_GEB5IAnF31IoTwNQNOF_bPDj8z at 3.1 MB/s, total 2.3 GB


In [30]:
!gdrive upload pretraining_REMBERT_79/final/config.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Uploading pretraining_REMBERT_79/final/config.json
Uploaded 1aDMz3s-HXnKBm9p6hA8rdHrQEL8rY2iL at 716.0 B/s, total 800.0 B
